<a href="https://colab.research.google.com/github/vitor0x5/simplified-shell/blob/master/soma_de_matrizes_com_threads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile somat.c

/* 
 ** Universidade Federal de São Carlos
 ** Departamento de Computação
 ** Prof. Hélio Crestana Guardia
 */

/*
    Programa : Programa com threads para soma de matrizes 
    Objetivo: programa em C que realiza a soma de 2 matrizes em paralelo,
              usando múltiplas threads., criadas com chamadas a funções da 
              biblioteca pthreads. 
*/

/*
    Autor: Vitor Hugo Guilherme RA: 744359
    Baseado no esqueleto de código feito pelo Professor Hélio Crestana Guardia
*/

/* 
    Comando para compilar: $ gcc somat.c -o somat
    Para executar: $ ./somat [-nl numero de linhas] [-nc numero de colunas] [-nt numero de threads] [-c multiplicador] [-v] verbose
*/

#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>

#define LEN 128

#define NLIN 2048
#define NCOL 2048
#define NTHR 4

int **_A;
int **_B;

int *thread_atual;
int n_linhas_por_thread;

int _nlin = 0;
int _ncol = 0;
int _nthr = 0;

int _verbose=0;

int _const = 1;

void * calcula(void * arg){
    int *thread_atual_t = (int *) arg;  // numero da thread atual
    int i, j;

    // linhas que a thread irá processar
    int start = *thread_atual_t * n_linhas_por_thread;
    int end = start + n_linhas_por_thread;

    for (i = start; i < end; i++) {
        for (j = 0; j < _ncol; j++) {
            _B[i][j] = _const * _A[i][j] + _B[i][j];
        }
    }
 
    return NULL;
}

int main (int argc, char *argv[]) {
    pthread_t *threads;
    int status;
    long t;
    char err_msg[LEN];
    int i,j,ind;

    ind=1;
    while (ind<argc) {
        
        if( !strcmp(argv[ind],"-h") || !strcmp(argv[ind],"/?")) {
        printf("Uso: %s [-nc num_col] [-nl num_lin] [-nt num_thr] "
                "[-c const] // valor para as multiplicações  [-v] // verbose\n",argv[0]);
        exit(0);
        }
        if(!strcmp(argv[ind],"-v"))
        _verbose=1;

        if(!strcmp(argv[ind],"-c")) {
        if(argc>ind)
            _const=atoi(argv[++ind]);
        else {
            printf("Erro nos parametros...\n");
            exit(0);
        }
        }
        if(!strcmp(argv[ind],"-nc")) {
        if(argc>ind)
            _ncol=atoi(argv[++ind]);
        else {
            printf("Erro nos parametros...\n");
            exit(0);
        }
        }
        if(!strcmp(argv[ind],"-nl")) {
        if(argc>ind)
            _nlin=atoi(argv[++ind]);
        else {
            printf("Erro nos parametros...\n");
            exit(0);
        }
        }
        if(!strcmp(argv[ind],"-nt")) {
        if(argc>ind)
            _nthr=atoi(argv[++ind]);
        else {
            printf("Erro nos parametros...\n");
            exit(0);
        }
        }
        ind++;
    }

    if(_nlin==0)
      _nlin = NLIN;
    if(_ncol==0)
      _ncol = NCOL;
    if(_nthr==0)
      _nthr = NTHR;

    // alocar as matrizes
    _A=(int **)malloc(_nlin*sizeof(int*));
    _B=(int **)malloc(_nlin*sizeof(int*));
    for(i=0;i<_nlin;i++) {
        _A[i]=(int *)malloc(_ncol*sizeof(int));
        _B[i]=(int *)malloc(_ncol*sizeof(int));
    }
    // alocação também pode ser feita na forma de vetor unidimensional: nlin * ncol * sizeof (int)

    // inicia semente de números aleatórios
    srand(getpid());

    // geração das matrizes com valores aleatórios. Senão, 0!
    for(i=0;i<_nlin;i++)
        for(j=0;j<_ncol;j++) {
            _A[i][j]=rand()%10;
            _B[i][j]=rand()%10;
        }

    if(_verbose) {
            printf("\n");
        for(i=0;i<_nlin;i++) {
            printf("%d: ",i);
        for(j=0;j<_ncol;j++)
            printf("%d ",_A[i][j]);
        printf("\t");
        for(j=0;j<_ncol;j++)
            printf("%d ",_B[i][j]);
        printf("\n");
        }
    }

    // alocar espaco para os identificadores das threads
    threads = (pthread_t *) malloc(_nthr*sizeof(pthread_t));

    if(_verbose) {
        printf("Processo %d vai criar threads...\n\n", getpid() );
    }

    // aloca o vetor de threads atuais
    // cada posição desse vetor irá armazenar um número referente a uma thread [0, 1, 2.._nthr]
    thread_atual = (int *)malloc(_nthr *sizeof(int));

    // Numero de linhas que cada thread irá processar
    n_linhas_por_thread = _nlin/_nthr;

    // cria threads
    for (t=0; t < _nthr; t++) {
        thread_atual[t] = t;    // Apenas uma posição desse vetor é passado para cada thread
                                // dessa forma, cada thread acessa seu numero corretamente


        // Realiza a multiplicação dos elementos pelo valor constante, e as somas das 
        // linhas, dividindo as linhas para cálculo em paralelo por múltiplas threads
		status = pthread_create(&threads[t], NULL, calcula, (void *)&thread_atual[t]);

        if (_verbose) {
            if (!status) {
                printf("main criou thread %d (%lu)\n",t,(long unsigned)threads[t]);
            } else {
                printf("Falha da criacao da thread %d (%d)\n",t,status);
                return (1);
            }
        }
    }

    int result;

    // espera pelas outras threads, sincronizando-as antes de imprimir os resultados
    for(t=0; t<_nthr; t++) {
		if((result=pthread_join(threads[t], NULL))) {
			strerror_r(result,err_msg,LEN);
			printf("Erro em pthread_join: %s\n",err_msg);
			exit(0);
		}
	}
    
 
  // impressão da matriz resultante
  if(_verbose) {
    // printf("\n");
    for(i=0;i<_nlin;i++) {
      printf("%d: ",i);
      for(j=0;j<_ncol;j++)
        printf("%2d ",_B[i][j]);
      printf("\n");
    }
    printf("\n");
  }

  // desalocar as matrizes
  for(i=0;i<_nlin;i++) {
    free(_A[i]);
    free(_B[i]);
  }
  free(_A);
  free(_B);

  return(0);
}

Writing somat.c


In [2]:
! gcc -Wall somat.c -o somat -pthread
! ./somat -nc 12 -nl 12 -nt 4 -c 2 -v

somat.c: In function ‘main’:
somat.c:186:44: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
                 printf("main criou thread %d (%lu)\n",t,(long unsigned)threads[t]);
                                           ~^
                                           %ld
somat.c:188:53: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
                 printf("Falha da criacao da thread %d (%d)\n",t,status);
                                                    ~^
                                                    %ld

0: 0 4 9 6 1 2 0 9 4 2 2 5 	7 1 1 7 5 6 5 9 6 5 2 1 
1: 9 2 8 8 3 2 6 1 5 5 9 7 	5 8 6 8 4 4 0 9 4 6 7 5 
2: 3 2 3 0 4 4 1 3 9 0 7 6 	9 7 7 4 3 2 4 6 7 7 4 3 
3: 8 1 1 6 9 2 8 9 9 7 0 6 	2 9 8 4 9 4 4 3 5 0 0 9 
4: 7 8 3 2 8 2 3 6 9 8 4 5 	9 7 4 3 3 9 0 2 0 0 9 4 
5: 5 4 5 6 4 9 8 0 6 8 4 3 	2 5 3 2 4 7 1 9 4 9 4 6 
6: 6 6 3 5 5 5 6 7 7 7 0 7 	4 1 3 0 1 2 1 0 8 8 9 8 
7: 3